In [1]:
# Импорты, подключение к базe
import cx_Oracle
import csv
import pandas as pd
import hashlib

con = cx_Oracle.connect(user='student_kuzevol', password='student_kuzevol',
                        dsn='79.164.32.101:1522/orcl')
cur = con.cursor()

### Пользователи

In [2]:
# Загрузка данных о пользователях
data = pd.read_csv('USERS_pr.csv')
data = data.fillna('')

sql =  f'''insert into users (
            person,
            current_age,          
            retirement_age,         
            birth_year,             
            birth_month,               
            gender,                    
            address,                 
            apartment,               
            city,                     
            state,                    
            zipcode,                  
            latitude,               
            longitude,              
            per_capita_income_zipcode,
            yearly_income_person,   
            total_debt,                
            fico_score,            
            num_credit_cards)
            values (
            :1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, SUBSTR(:14, 2), SUBSTR(:15, 2), SUBSTR(:16, 2), :17, :18
            )'''

for i,row in data.iterrows():
    try:
        cur.execute(sql, tuple(row))
    except Exception as e:
        print(e)
        print(row)
con.commit()

### Карты

In [3]:
# Загрузка данных о картах
# Для поля expires в формате mm/yyyy берется последнее число месяца
sql = f'''insert into cards (
            person_id,
            card_index,
            card_brand,
            card_type,
            dpan,
            expires,
            cvv,
            has_chip,
            cards_issued,
            credit_limit,
            acct_open_date,
            year_pin_last_changed
            )
            values (:1, :2, :3, :4, :5, LAST_DAY(TO_DATE(:6,'mm/yyyy')), :7, :8, :9, SUBSTR(:10, 2), TO_DATE(:11, 'mm/yyyy'), :12)
'''
with open('CARDS_pr.csv', 'r') as csv_file:
    card_dpan = {}
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for row in csv_reader:
        try:
            hash_str = row[4][:6]+hashlib.md5(row[4][6:len(row[4])-4].encode()).hexdigest()+row[4][len(row[4])-4:]
            cur.execute(sql, (row[0], row[1], row[2], row[3], hash_str, row[5], row[6], row[7], row[8], row[9], row[10], row[11]))
            card_dpan[row[4]] = hash_str
        except Exception as e:
            print(e)
            print(row)
con.commit()

In [4]:
# Загрузка справочника карта/dpan
sql = f'''insert into card_dpan (
            card_number,
            dpan
            )
            values (:1, :2)
'''

for key in card_dpan:
    try:
        cur.execute(sql, (key, card_dpan[key]))
    except Exception as e:
        print(e)
        print(row)
con.commit()

### Транзакции

In [2]:
# Транзакции

# Функция для корректировки исходных данных
def fix_line(bad_line):
    new_line=bad_line[:len(bad_line)-2]
    new_line=new_line+[', '.join(bad_line[len(bad_line)-2:])]
    return new_line

trans_2018 = pd.read_csv('CCT_2018_1_pr.csv',
                        dtype={'ZIP':str}).drop_duplicates(ignore_index=True)
trans_2019 = pd.read_csv('CCT_2019_1_pr.csv', on_bad_lines=fix_line, engine='python',
                        dtype={'ZIP':str}).drop_duplicates(ignore_index=True)
trans_2020 = pd.read_csv('CCT_2020_1_pr.csv', on_bad_lines=fix_line, engine='python',
                        dtype={'ZIP':str}).drop_duplicates(ignore_index=True)

In [3]:
# Корректировка исходных данных
cols=list(trans_2020.columns)
df = pd.merge(trans_2019,trans_2020, how='outer', indicator = True)
trans_2020=df[cols][df['_merge']=='right_only']

In [4]:
# Формат данных для cur.executemany(...)
trans_2018 = trans_2018.where(pd.notnull(trans_2018), None)
trans_2018 = list(trans_2018.itertuples(index=False, name=None))
trans_2019 = trans_2019.where(pd.notnull(trans_2019), None)
trans_2019 = list(trans_2019.itertuples(index=False, name=None))
trans_2020 = trans_2020.where(pd.notnull(trans_2020), None)
trans_2020 = list(trans_2020.itertuples(index=False, name=None))

In [5]:
# Статистика
print(len(trans_2018))
print(len(trans_2019))
print(len(trans_2020))

1721612
1723942
336500


In [7]:
insert_query =  f'''insert into transactions (
                transaction_time,
                person_id,
                card,
                amount,
                use_chip,
                merchant_name,
                merchant_city,
                merchant_state,
                zip,
                mcc,
                err
                )
                values (
                to_date(substr(:1, 1, 19), 'yyyy-mm-dd hh24:mi:ss'), :2, :3, :4, :5, :6, :7, :8, substr(:9, 1, 5), :10, :11
                )'''

In [10]:
# 2018
try:
    cur.executemany(insert_query, trans_2018)
    con.commit()
except Exception as e:
    print(e)

In [14]:
# 2019
try:
    cur.executemany(insert_query, trans_2019)
    con.commit()
except Exception as e:
    print(e)

In [8]:
# 2020
try:
    cur.executemany(insert_query, trans_2020)
    con.commit()
except Exception as e:
    print(e)

In [9]:
# Закрытие подключения
cur.close()
con.close()